In [1]:
abbie_list = [
    "708367",
    "754614",
    "708371",
    "708364",
    "782745",
    "782747",
    "747810",
    "747813",
    "717442",
    "747806",
    "715344",
    "754609",
    "792097",
    "758788",
    "719651",
    "737561",
    "720811",
    "729504",
    "789029",
    "789031",
    "789033",
    "721826",
    "730901",
    "747822",
    "754616",
    "745317",
    "745318",
    "782646",
    "782738",
]

In [2]:
my_list = [
    "692912",  
    "716946",
    "716947",
    "716948",
    "716949",
    "716950",
    "716951",
    "725231",
    "725328",
    "725329",
    "728854",
    "731907",
    "737038",
    "737042",
    "744117",
    "744119",
    "746041",
    "746043",
    "746045",
    "746046",
    "751017",
    "751019",
    "751023",
    "751024",
    "751035",
    "755069",
    "755071",
    "755072",
    "755073",
    "755790",
    "762196",
    "762199",
]

In [3]:
abbie_set = set(abbie_list)
my_set = set(my_list)
overlap = abbie_set & my_set
print(overlap)

set()


In [4]:
# Query for all SmartSPIM assets to find unique subjects
from aind_data_access_api.document_db import MetadataDbClient

# Initialize the database client
client = MetadataDbClient(
    host="api.allenneuraldynamics.org",
    database="metadata_index", 
    collection="data_assets"
)

# Query for all records with "SmartSPIM" in the name
smartspim_records = client.retrieve_docdb_records(
    filter_query={"name": {"$regex": "SmartSPIM", "$options": "i"}},
    projection={"subject.subject_id": 1, "name": 1}
)

print(f"Total SmartSPIM records found: {len(smartspim_records)}")

# Extract unique subject IDs
unique_subjects = set()
for record in smartspim_records:
    if 'subject' in record and 'subject_id' in record['subject']:
        unique_subjects.add(record['subject']['subject_id'])

print(f"Number of unique subjects with SmartSPIM assets: {len(unique_subjects)}")
print(f"Unique subject IDs: {sorted(list(unique_subjects))}")

Total SmartSPIM records found: 5337
Number of unique subjects with SmartSPIM assets: 2407
Unique subject IDs: ['156631', '18559', '19603', '3283906', '3283909', '393', '394', '397', '568105', '595922', '597305', '603261', '605078', '605079', '607463', '608846', '610879', '611328', '611334', '612962', '613813', '614952', '614956', '619778', '620326', '620633', '620642', '620643', '620796', '621362', '622280', '622386', '622464', '622466', '622468', '622638', '622640', '622957', '623705', '623707', '623710', '623711', '623894', '623900', '624643', '624644', '624645', '624646', '625098', '625099', '625100', '625382', '625463', '625464', '625749', '626182', '626185', '627137', '628286', '628289', '628290', '631680', '631709', '631710', '631977', '631978', '632268', '632521', '633190', '634568', '634569', '634571', '638312', '638314', '638315', '638316', '638665', '638779', '638781', '638927', '638929', '639090', '639091', '639092', '639374', '639378', '639485', '640387', '640390', '640391'

In [15]:
# Load Excel spreadsheet data
import sys
import pandas as pd
sys.path.append('/Users/doug.ollerenshaw/code/aind-workbench/projects/patchseq_metadata')

from excel_loader import load_specimen_procedures_excel

# Load the spreadsheet data
print("Loading Excel spreadsheet data...")
sheet_data = load_specimen_procedures_excel()

# Extract unique subject IDs from the Batch Info sheet
spreadsheet_subjects = set()
if sheet_data and "Batch Info" in sheet_data:
    batch_info = sheet_data["Batch Info"]
    for idx, row in batch_info.iterrows():
        lab_track_ids = str(row['LabTrack IDs']) if pd.notna(row['LabTrack IDs']) else ""
        # Split by comma and clean up whitespace
        if lab_track_ids and lab_track_ids != 'nan':
            # Split by comma, strip whitespace, and filter out empty strings
            subjects = [s.strip() for s in lab_track_ids.split(',') if s.strip()]
            for subject in subjects:
                # Only add if it's all digits (valid subject ID)
                if subject.isdigit():
                    spreadsheet_subjects.add(subject)

print(f"Found {len(spreadsheet_subjects)} unique subjects in spreadsheet")
print(f"Sample subjects: {sorted(list(spreadsheet_subjects))[:10]}")

Loading Excel spreadsheet data...
Loading Excel file: DT_HM_TissueClearingTracking_.xlsx
Found 14 total sheets

--- Loading 'Batch Info' sheet ---
✓ Batch Info loaded - Shape: (160, 5)
  Columns: ['Batch #', 'LabTrack IDs', 'Notes', 'Total #', 'Column_4']

--- Loading 'Dec 2022 - Feb 2023' sheet ---
✓ Dec 2022 - Feb 2023 loaded - Shape: (34, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Mar - May 2023' sheet ---
✓ Mar - May 2023 loaded - Shape: (45, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Jun - Aug 2023' sheet ---
✓ Jun - Aug 2023 loaded - Shape: (54, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'ClearingMethod', 'Experimenter', 'Institute', 'Fixation_SHIELD OFF_Date(s)']...

--- Loading 'Sep - Nov 2023' sheet ---
✓ Sep - Nov 2023 loaded - Shape: (32, 25)
  Sample columns: ['SubjInfo_Mouse ID', 'Cl

In [14]:
import pandas as pd

# Get all SmartSPIM records with the fields we need
print("Querying database for all SmartSPIM records with procedures...")

smartspim_with_procedures = client.retrieve_docdb_records(
    filter_query={"name": {"$regex": "SmartSPIM", "$options": "i"}},
    projection={
        "name": 1, 
        "subject.subject_id": 1, 
        "procedures.specimen_procedures": 1,
        "data_description.project_name": 1,
        "data_description.institution.abbreviation": 1,
    }
)

print(f"Retrieved {len(smartspim_with_procedures)} SmartSPIM records")

# Process each record
data = []
for record in smartspim_with_procedures:
    # Extract subject ID
    subject_id = record.get('subject', {}).get('subject_id')
    if not subject_id:
        continue
    
    # Extract asset name
    asset_name = record.get('name', 'Unknown')
    
    # Extract collection date from asset name
    collection_date = "Unknown"
    try:
        parts = asset_name.split('_')
        for part in parts:
            if len(part) == 10 and part.count('-') == 2:  # YYYY-MM-DD format
                collection_date = part
                break
    except:
        pass
    
    # Check specimen_procedures
    has_specimen_procedures = False
    procedures = record.get('procedures', {})
    if procedures:
        specimen_procedures = procedures.get('specimen_procedures', [])
        has_specimen_procedures = len(specimen_procedures) > 0
    
    # Check if mouse is in spreadsheet
    in_spreadsheet = str(subject_id) in spreadsheet_subjects
    
    data.append({
        'mouse_id': subject_id,
        'collection_date': collection_date,
        'has_specimen_procedures': has_specimen_procedures,
        'asset_name': asset_name,
        'project_name': record.get('data_description', {}).get('project_name', 'Unknown'),
        'institution': record.get('data_description', {}).get('institution', {}).get('abbreviation', 'Unknown'),
        'in_spreadsheet': in_spreadsheet
    })

# Create DataFrame
df = pd.DataFrame(data)

# Remove duplicates (keep first occurrence for each mouse)
df_unique = df.drop_duplicates(subset=['mouse_id'], keep='first')

# Sort by collection date
df_unique = df_unique.sort_values('collection_date')

df_unique

Querying database for all SmartSPIM records with procedures...
Retrieved 5337 SmartSPIM records
Retrieved 5337 SmartSPIM records


,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
2504,620796,2022-05-31,False,SmartSPIM_620796_2022-05-31_17-54-53,None,Unknown,False
870,622640,2022-05-31,False,SmartSPIM_622640_2022-05-31_12-01-30,None,Unknown,False
4924,622638,2022-06-07,False,SmartSPIM_622638_2022-06-07_15-33-13,None,Unknown,False
880,622957,2022-06-07,False,SmartSPIM_622957_2022-06-07_10-54-57,None,Unknown,False
2459,619778,2022-07-19,False,SmartSPIM_619778_2022-07-19_13-57-15_stitched_...,None,Unknown,False
...,...,...,...,...,...,...,...
3890,807684,2025-07-30,False,SmartSPIM_807684_2025-07-30_19-07-22,BG AAV Toolbox,AIBS,False
3892,807687,2025-07-30,False,SmartSPIM_807687_2025-07-30_22-44-06,BG AAV Toolbox,AIBS,False
3893,777096,2025-07-30,False,SmartSPIM_777096_2025-07-30_18-44-50,Cell Type LUT,AIND,True
4560,807695,2025-07-30,False,SmartSPIM_807695_2025-07-30_14-29-09,BG AAV Toolbox,AIBS,False


In [8]:
df_unique = df_unique.sort_values('collection_date')
df_unique

,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
2504,620796,2022-05-31,False,SmartSPIM_620796_2022-05-31_17-54-53,None,Unknown,False
870,622640,2022-05-31,False,SmartSPIM_622640_2022-05-31_12-01-30,None,Unknown,False
4924,622638,2022-06-07,False,SmartSPIM_622638_2022-06-07_15-33-13,None,Unknown,False
880,622957,2022-06-07,False,SmartSPIM_622957_2022-06-07_10-54-57,None,Unknown,False
2459,619778,2022-07-19,False,SmartSPIM_619778_2022-07-19_13-57-15_stitched_...,None,Unknown,False
...,...,...,...,...,...,...,...
4560,807695,2025-07-30,False,SmartSPIM_807695_2025-07-30_14-29-09,BG AAV Toolbox,AIBS,False
3879,807681,2025-07-30,False,SmartSPIM_807681_2025-07-30_15-55-38,CTY Genetic Tools,AIBS,False
3894,807696,2025-07-30,False,SmartSPIM_807696_2025-07-30_17-15-30,BG AAV Toolbox,AIBS,False
3890,807684,2025-07-30,False,SmartSPIM_807684_2025-07-30_19-07-22,BG AAV Toolbox,AIBS,False


In [9]:
df_unique['has_specimen_procedures'].value_counts()

has_specimen_procedures
False    1513
True      894
Name: count, dtype: int64

In [10]:
df_unique.query('has_specimen_procedures == True').tail(25)

,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
703,713442,2024-12-05,True,SmartSPIM_713442_2024-12-05_10-22-57_stitched_...,Ephys Platform,AIND,False
3002,774922,2024-12-09,True,SmartSPIM_774922_2024-12-09_13-20-56,Discovery-Brain Wide Circuit Dynamics,AIND,False
3401,774923,2024-12-10,True,SmartSPIM_774923_2024-12-10_10-49-57_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
3018,774925,2024-12-13,True,SmartSPIM_774925_2024-12-13_17-18-14_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
3017,774924,2024-12-13,True,SmartSPIM_774924_2024-12-13_09-02-38_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
3399,774926,2024-12-16,True,SmartSPIM_774926_2024-12-16_10-16-15_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
2464,774927,2024-12-16,True,SmartSPIM_774927_2024-12-16_18-03-58_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
3404,774928,2024-12-17,True,SmartSPIM_774928_2024-12-17_17-41-54_stitched_...,Discovery-Brain Wide Circuit Dynamics,AIND,False
935,744744,2025-02-12,True,SmartSPIM_744744_2025-02-12_14-07-01_stitched_...,Thalamus in the middle - Project 4 Frontal cor...,AIND,False
3193,764791,2025-02-13,True,SmartSPIM_764791_2025-02-13_10-49-11_stitched_...,Thalamus in the middle - Project 4 Frontal cor...,AIND,False


In [11]:
df_unique.query('in_spreadsheet == True').tail(25)

,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
4444,761154,2025-07-23,False,SmartSPIM_761154_2025-07-23_16-22-16,Brain Computer Interface,AIND,True
4431,781470,2025-07-23,False,SmartSPIM_781470_2025-07-23_22-49-43,Thalamus in the middle - Project 4 Frontal cor...,AIND,True
4448,780176,2025-07-24,False,SmartSPIM_780176_2025-07-24_13-03-06,Cell Type LUT,AIND,True
4439,796303,2025-07-24,False,SmartSPIM_796303_2025-07-24_11-27-10,Neurobiology of Action,AIND,True
4430,802583,2025-07-24,False,SmartSPIM_802583_2025-07-24_01-07-30,Thalamus in the middle - Project 1 Mesoscale t...,AIND,True
4452,796306,2025-07-24,False,SmartSPIM_796306_2025-07-24_18-47-51,Neurobiology of Action,AIND,True
4454,780175,2025-07-24,False,SmartSPIM_780175_2025-07-24_19-48-01,Cell Type LUT,AIND,True
4443,796307,2025-07-24,False,SmartSPIM_796307_2025-07-24_15-14-17,Neurobiology of Action,AIND,True
3928,764642,2025-07-25,False,SmartSPIM_764642_2025-07-25_12-30-24,Genetic Perturbation Platform,AIND,True
4457,773682,2025-07-25,False,SmartSPIM_773682_2025-07-25_02-41-58,Cell Type LUT,AIND,True


In [12]:
df_unique.query('has_specimen_procedures == False').tail(25)

,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
4506,805111,2025-07-26,False,SmartSPIM_805111_2025-07-26_13-16-23,BRAIN CONNECTS TransNeuronal Tools,AIBS,False
4504,805114,2025-07-26,False,SmartSPIM_805114_2025-07-26_14-16-38,BRAIN CONNECTS TransNeuronal Tools,AIBS,False
4503,805110,2025-07-26,False,SmartSPIM_805110_2025-07-26_10-30-37,BRAIN CONNECTS TransNeuronal Tools,AIBS,False
4508,805112,2025-07-26,False,SmartSPIM_805112_2025-07-26_16-02-26,BRAIN CONNECTS TransNeuronal Tools,AIBS,False
4483,805113,2025-07-26,False,SmartSPIM_805113_2025-07-26_11-31-44,BRAIN CONNECTS TransNeuronal Tools,AIBS,False
4522,777271,2025-07-28,False,SmartSPIM_777271_2025-07-28_18-03-41,Cell Type LUT,AIND,True
4520,802167,2025-07-28,False,SmartSPIM_802167_2025-07-28_20-06-01,CTY Genetic Tools,AIBS,False
4517,807694,2025-07-28,False,SmartSPIM_807694_2025-07-28_16-26-36,CTY Genetic Tools,AIBS,False
4515,777098,2025-07-28,False,SmartSPIM_777098_2025-07-28_11-37-52,Cell Type LUT,AIND,True
4463,802734,2025-07-28,False,SmartSPIM_802734_2025-07-28_16-49-58,BG AAV Toolbox,AIBS,False


In [22]:
df_unique.query('has_specimen_procedures == False and institution == "AIND" and in_spreadsheet == True').tail(20)

,mouse_id,collection_date,has_specimen_procedures,asset_name,project_name,institution,in_spreadsheet
4448,780176,2025-07-24,False,SmartSPIM_780176_2025-07-24_13-03-06,Cell Type LUT,AIND,True
4439,796303,2025-07-24,False,SmartSPIM_796303_2025-07-24_11-27-10,Neurobiology of Action,AIND,True
4430,802583,2025-07-24,False,SmartSPIM_802583_2025-07-24_01-07-30,Thalamus in the middle - Project 1 Mesoscale t...,AIND,True
4452,796306,2025-07-24,False,SmartSPIM_796306_2025-07-24_18-47-51,Neurobiology of Action,AIND,True
4454,780175,2025-07-24,False,SmartSPIM_780175_2025-07-24_19-48-01,Cell Type LUT,AIND,True
4443,796307,2025-07-24,False,SmartSPIM_796307_2025-07-24_15-14-17,Neurobiology of Action,AIND,True
3928,764642,2025-07-25,False,SmartSPIM_764642_2025-07-25_12-30-24,Genetic Perturbation Platform,AIND,True
4457,773682,2025-07-25,False,SmartSPIM_773682_2025-07-25_02-41-58,Cell Type LUT,AIND,True
4460,767462,2025-07-25,False,SmartSPIM_767462_2025-07-25_17-25-14,Genetic Perturbation Platform,AIND,True
4461,761434,2025-07-25,False,SmartSPIM_761434_2025-07-25_12-03-36,Brain Computer Interface,AIND,True


In [13]:
import plotly.express as px
import pandas as pd
from datetime import datetime

# Convert collection_date to datetime for better plotting
df_plot = df_unique.copy()
df_plot = df_plot[df_plot['collection_date'] != 'Unknown']  # Remove unknown dates
df_plot['collection_date_dt'] = pd.to_datetime(df_plot['collection_date'])

# Sort by date
df_plot = df_plot.sort_values('collection_date_dt')

# Convert boolean to string for better hover display
df_plot['procedures_status'] = df_plot['has_specimen_procedures'].map({True: 'Has Procedures', False: 'No Procedures'})

# Create interactive scatter plot
fig = px.scatter(
    df_plot, 
    x='collection_date_dt', 
    y='has_specimen_procedures',
    color='procedures_status',
    hover_data=['mouse_id', 'collection_date', 'asset_name'],
    labels={
        'collection_date_dt': 'Collection Date',
        'has_specimen_procedures': 'Has Specimen Procedures',
        'mouse_id': 'Mouse ID',
        'collection_date': 'Collection Date',
        'asset_name': 'Asset Name'
    },
    title='SmartSPIM Specimen Procedures Over Time (Interactive)',
    color_discrete_map={'Has Procedures': 'red', 'No Procedures': 'blue'}
)

# Update layout
fig.update_layout(
    width=1200,
    height=600,
    yaxis=dict(
        tickmode='array',
        tickvals=[0, 1],
        ticktext=['False', 'True']
    )
)

# Update marker size and opacity
fig.update_traces(marker=dict(size=8, opacity=0.7))

fig.show()

print(f"Plotting {len(df_plot)} subjects with known collection dates")
print(f"Date range: {df_plot['collection_date_dt'].min()} to {df_plot['collection_date_dt'].max()}")

Plotting 2407 subjects with known collection dates
Date range: 2022-05-31 00:00:00 to 2025-07-30 00:00:00
